# Лабораторная работа 2. Классификация текстов на основе вхождения в документ словарных слов

**Задание 1.** Загрузите в датафрейм новостной датасет `lenta_ru_news_filtered.csv`, собранный на базе корпуса `lenta.ru v1.0`. В датасете каждая новость описывается следующими полями:
* **url** - адрес новости на сайте `lenta.ru`,
* **topic** - тема новости,
* **title** - заголовок новости,
* **text** - текст новости.

Ответьте на следуюшие вопросы:
1. Сколько всего новостных текстов?
2. На какие темы встречаются новости?
3. Сколько новостных текстов в каждой теме?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
news = pd.read_csv('/content/drive/MyDrive/NLP/labs/nlp_lab2/lenta_ru_news_filtered.csv')

In [4]:
news.head(5)

,Unnamed: 0,url,topic,title,text
0,34,https://lenta.ru/news/2018/12/15/putinrap/,Культура,Путин предостерег от запретов рэп-концертов,"Президент России Владимир Путин, выступая на з..."
1,52,https://lenta.ru/news/2018/12/15/cirk/,Культура,Путин пообещал поразбираться с Cirque du Solei...,Владимир Путин пообещал разобраться с открытие...
2,53,https://lenta.ru/news/2018/12/15/asomov/,Культура,Умер актер «Кривого зеркала»,В Ташкенте скончался участник юмористической т...
3,57,https://lenta.ru/news/2018/12/14/zinchuk/,Культура,Зинчук рассказал подробности аварии и призвал ...,"Гитарист-виртуоз и композитор Виктор Зинчук, г..."
4,63,https://lenta.ru/news/2018/12/14/bydlo/,Культура,За сравнившего с быдлом зрителей актера «Униве...,"Коллеги актера Андрея Гайдуляна, известного по..."


In [5]:
news.tail(5)

,Unnamed: 0,url,topic,title,text
3995,19915,https://lenta.ru/news/2018/07/09/zakladnaya/,Экономика,Отсутствие подзаконных актов помешало банкам р...,Российские банки пока не получили возможности ...
3996,19931,https://lenta.ru/news/2018/07/09/tram/,Экономика,Россиянам представили «секретный» трамвай,"«Уралвагонзавод» (УВЗ), входящий в состав «Рос..."
3997,19935,https://lenta.ru/news/2018/07/09/poroh/,Экономика,Украина захотела спасти Европу от «Северного п...,"Президент Украины Петр Порошенко считает, что ..."
3998,19941,https://lenta.ru/news/2018/07/09/cb_like_fcb/,Экономика,Центробанку разрешат проводить обыски,Банк России может получить разрешение беспрепя...
3999,19944,https://lenta.ru/news/2018/07/09/vodka/,Экономика,Россиян оставили без дешевой водки,Минфин уравнял минимальную розничную цену (МРЦ...


In [6]:
uniq_topic = news['topic'].unique()
uniq_topic

array(['Культура', 'Экономика'], dtype=object)

In [16]:
print("1. Всего новостных текстов:", len(news))

1. Всего новостных текстов: 4000


In [22]:
print("2. Темы новостей и количество статей в темах:")
for item in uniq_topic:
  print(f'Тема {item} - число статей: {len(news[news["topic"]==item])}')

2. Темы новостей и количество статей в темах:
Тема Культура - число статей: 2000
Тема Экономика - число статей: 2000


**Задание 2.** Выполните предобработку новостных текстов в виде:
- приведение к нижнему регистру,
- удаление знаков пунктуации.

**(!) Далее в лабораторной работе задания выполняются с полученными обработанными текстами**.

Разделите датасет на обучающую и тестовую части в соотношении 80% к 20%. Выведите диаграммы, отражающие количество текстов по каждой теме в каждой из частей.

**Задание 3.** Подсчитайте частоту встречаемости слов предобработанных новостных текстов `обучающей` части датафрейма. Какие слова употребляются наиболее часто вцелом в этих новостных текстах, а какие слова употребляются в этих же новостных текстах относительно тем (выведите топ-`50` слов для каждого случая)?

Нахождение частот слов в новостных текстах датафрейма можно выполнять посредством инструмента `FreqDist` библиотеки `NLTK`.

Например:
```
df['text'].apply(lambda x: nltk.FreqDist(nltk.word_tokenize(x)))
```

**Задание 4.** Составьте словари из текстов `обучающей` части датасетов наболее часто встречающихся слов, которые встречаются **только в одной из каждых тем**. Постройте диаграмму для топ-`50` этих слов для каждой темы (ось X - слова, ось Y - частоты встречаемости слов в новостных текстах).

**Задание 5.** Выполните классификацию новостных текстов из `тестовой` части датасета на основе `top-k слов` (`k` - как входной параметр) из словарей, построенных в задании `4` на основе `обучающей` части датасета. Оцените показатели `полнота` и `точность` классификации (относительно значения параметра `k`) при указании количества документов, для которых не получилось определить тему.

Общий алгоритм классификации:

- Для документа `x` считаем сколько встречается слов, из словаря каждой из тем.
- Выбираем тему для документа `x` с максимальным количеством слов
- Отдельно обрабатываем случаи, когда количество слов одинаковое или равно 0. Это случаи, когда не получилось определить тему документа.

**Задание 6.** Познакомьтесь с библиотекой Natasha для обработки текстов на русском языке, прочитав <a href="https://habr.com/ru/articles/516098/">статью</a>.

#### Импорт библиотек и создание объекта для работы с текстом
Импортируйте библиотеки для сегментации на предложения, морфологического и синтаксического анализа.

```python
from natasha import(
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc,
)
```

Создайте объект из обрабатываемого текста `text`:

```python
text_doc = Doc(text)
```

#### Сегментация на предложения
Создайте объект, который будет выполнять сегментацию текста на предложения:

```python
segmenter = Segmenter()
```

С помощью сегментатора можно разбить текст на предложения и токены:

```python
text_doc.segment(segmenter)
print(text_doc.tokens)
print(text_doc.sents)
```

#### Морфологический анализ

Инициализируйте словарь и морфологический анализатор:
```python
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)
```

Чтобы получить морфологическую разметку слова `word` воспользуйтесь:

```python
morph_vocab.parse(word)
```

Добавить морфологическую разметку к объекту обрабатываемого текста (Doc) можно как

```python
text_doc.tag_morph(morph_tagger)
print(text_doc.tokens)
text_doc.sents[0].morph.print()
```

Получить леммы текста можно через метод `lemmatize`:

```python
for token in text_doc.tokens:
    token.lemmatize(morph_vocab)
{_.text: _.lemma for _ in text_doc.tokens}
```

**Задание 7.** Добавьте столбец в обучающую и тестовую часть датасета с обработанными текстами после лемматизации.

**Задание 8.** Составьте словари на основе текстов `обучающей` части датасета по аналогии с **заданием 4.**, но с текстами, полученными после лемматзации.

**Задание 9.** Сделайте классификацию новостных текстов `тестовой` части датасета по аналогии с **заданием 5.**, но с текстами, полученными **после лемматзации**.